In [1]:
import numpy as np
import pandas as pd
import smatch

#If using script on terminal
#from tqdm import tqdm
from tqdm.notebook import tqdm

In [2]:
data = pd.read_csv('SDSS_catalog_hpix.csv')
print(len(data))
data.head()

41844


,recno,objID,M15,Pdisk,Pedgeon,PbarGZ2,PbarNair10,Pmerg,Pbulge,Pcigar,TType,PS0,RA,DE,HPIX_4096
0,562371,588015509289107593,562371,0.007895,8.908050e-03,0.004785,0.416281,0.025819,0.003938,0.284150,3.99326,0.999085,45.0749,0.11085,73
1,562372,588015509289107598,562372,0.000452,4.362040e-07,0.002019,0.348180,0.041967,0.079616,0.000327,1.98614,0.993579,45.0906,0.13660,79
2,562373,588015509289107620,562373,0.002187,9.806280e-01,0.009124,0.320715,0.019539,0.905458,0.999917,1.22539,0.973403,45.1526,0.17028,264
3,149629,587731513154273388,149629,0.002267,8.160070e-01,0.004812,0.288495,0.006018,0.000113,0.984632,3.29565,0.987963,45.2996,0.30291,345
4,149628,587731513154207938,149628,0.000280,7.518660e-02,0.054535,0.312281,0.084383,0.095057,0.006318,-0.38629,0.930315,45.2152,0.35570,466


In [3]:
catalog = pd.read_csv('des_matched.csv')
print(len(catalog))
catalog.head()

491186


,COADD_OBJECT_ID,TILENAME,HPIX_16384,HPIX_4096,DNF_ZMEAN_SOF,RA,DEC,FLUX_RADIUS_G,FLUX_RADIUS_R,FLUX_RADIUS_I,FLUX_RADIUS_Z,SOF_BDF_G_1,SOF_BDF_G_2,IMAFLAGS_ISO_G,IMAFLAGS_ISO_R,IMAFLAGS_ISO_I,IMAFLAGS_ISO_Z
0,1357042778,DES0259+0001,1183,73,1.087239,45.074356,0.119060,2.068099,2.430418,1.613238,1.160763,0.014225,0.030073,0,0,0,0
1,1357042880,DES0259+0001,1181,73,1.012709,45.080292,0.117607,2.859771,2.439775,2.373129,2.635680,-0.170219,-0.124105,0,0,0,0
2,1357042996,DES0259+0001,1177,73,0.044156,45.074895,0.110863,7.117305,7.002007,6.948651,6.879068,0.270914,-0.006935,0,0,0,0
3,1357043005,DES0259+0001,1175,73,1.689987,45.082463,0.116101,2.116432,1.593948,1.689177,1.618437,0.007744,-0.019421,0,0,0,0
4,1357043031,DES0259+0001,1181,73,1.055867,45.079527,0.115751,2.433353,1.587338,2.668203,2.444284,-0.035244,0.106001,0,0,0,0


In [4]:
ra1 = data['RA']
dec1 = data['DE']
ra2 = catalog['RA']
dec2 = catalog['DEC']

matches = smatch.match(ra1, dec1, 0.0003, ra2, dec2, nside=4096, maxmatch=1)
data = data.drop(labels = ['RA', 'DE', 'recno', 'objID', 'HPIX_4096'], axis=1)
print(len(matches))

17205


In [5]:
new_data = pd.DataFrame()
for i in tqdm(range(len(matches))):
    #if(i == 100): break
    point = pd.concat( (data.iloc[matches[i][0]], catalog.iloc[matches[i][1]]), axis=0)
    new_data = new_data.append(point, ignore_index=True)

  0%|          | 0/17205 [00:00<?, ?it/s]

In [6]:
new_data = new_data.reindex(columns=[ 'COADD_OBJECT_ID', 'TILENAME', 'HPIX_16384', 'HPIX_4096', 'DNF_ZMEAN_SOF', 
                                     'RA', 'DEC', 'M15', 
        'Pdisk', 'Pedgeon', 'PbarGZ2', 'PbarNair10', 'Pmerg', 'Pbulge', 'Pcigar', 'TType', 'PS0', 'FLUX_RADIUS_G', 
        'FLUX_RADIUS_R', 'FLUX_RADIUS_I', 'FLUX_RADIUS_Z', 'SOF_BDF_G_1', 'SOF_BDF_G_2', 'IMAFLAGS_ISO_G', 
        'IMAFLAGS_ISO_R', 'IMAFLAGS_ISO_I', 'IMAFLAGS_ISO_Z' ])

new_data = new_data.astype({'TILENAME': str, 'COADD_OBJECT_ID': int,'HPIX_16384': int, 'HPIX_4096': int})

In [7]:
new_data.head()

,COADD_OBJECT_ID,TILENAME,HPIX_16384,HPIX_4096,DNF_ZMEAN_SOF,RA,DEC,M15,Pdisk,Pedgeon,...,FLUX_RADIUS_G,FLUX_RADIUS_R,FLUX_RADIUS_I,FLUX_RADIUS_Z,SOF_BDF_G_1,SOF_BDF_G_2,IMAFLAGS_ISO_G,IMAFLAGS_ISO_R,IMAFLAGS_ISO_I,IMAFLAGS_ISO_Z
0,1357042996,DES0259+0001,1177,73,0.044156,45.074895,0.110863,562371.0,0.007895,8.908050e-03,...,7.117305,7.002007,6.948651,6.879068,0.270914,-0.006935,0.0,0.0,0.0,0.0
1,1357040990,DES0259+0001,1265,79,0.097686,45.090614,0.136579,562372.0,0.000452,4.362040e-07,...,10.188334,10.108422,9.822471,9.754218,0.016191,0.090772,0.0,0.0,0.0,0.0
2,1357037576,DES0259+0001,4224,264,0.132932,45.152588,0.170296,562373.0,0.002187,9.806280e-01,...,8.861989,7.905922,7.487818,7.119528,0.247162,0.491628,0.0,0.0,0.0,0.0
3,1356894408,DES0259+0001,7463,466,0.042477,45.215201,0.355711,149628.0,0.000280,7.518660e-02,...,10.586360,10.611771,10.462768,10.749476,0.216854,-0.011633,0.0,0.0,0.0,0.0
4,1356899664,DES0259+0001,8568,535,0.185637,44.885581,0.231764,149625.0,0.003596,3.137700e-01,...,23.625998,21.879747,21.336260,21.204153,-0.006813,0.192957,0.0,0.0,0.0,0.0


In [9]:
new_data.to_csv('morphological_catalog.csv', index=False)